In [ ]:
!pip install tensorflow-hub

In [ ]:
!pip install tensorflow-addons

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, utils
import tensorflow_addons as tfa

ModuleNotFoundError: No module named 'keras.src.engine'

In [ ]:
import numpy as np
import tensorflow as tf
print(f"tensorflow version: {tf.__version__}")
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer, Conv2D, MaxPooling2D

import matplotlib.pyplot as plt

In [ ]:
# pip install tensorflow==2.11.1

In [ ]:
import json
import math
import os
import statistics
from keras import layers
from keras import backend as K
from keras.models import Sequential
from keras.preprocessing import image
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.layers import Input, Dense, DepthwiseConv2D,AveragePooling2D, Concatenate, Dropout, Permute,Reshape,Lambda,Activation, Add,Multiply, MaxPooling2D, Conv2D, Flatten, BatchNormalization, GlobalAveragePooling2D,LayerNormalization

import tensorflow as tf
from tensorflow.keras.applications import VGG16,ConvNeXtTiny,ResNet50, MobileNet, Xception, EfficientNetB0 , DenseNet169, DenseNet201, DenseNet121, InceptionV3, NASNetLarge, InceptionResNetV2, NASNetMobile
from tensorflow.keras.optimizers.legacy import Adam, SGD
from tensorflow.keras.metrics import Precision, Recall, AUC

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
from keras.models import Model
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import gc
import itertools
from tqdm import tqdm
from PIL import Image
from functools import partial
from collections import Counter
from statistics import mean

from keras.models import load_model
#from keras.models import Sequential
from matplotlib import pyplot as plt
import h5py
import cv2
import glob



base_data_dir = ''
out_loc = os.path.join(base_data_dir, '/content/drive/MyDrive/')

%matplotlib inline


In [ ]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
mag = '40'
# benign = '/content/drive/MyDrive/BreakHis_data/40/B'
# malignant = '/content/drive/MyDrive/BreakHis_data/40/M'
A = '/content/drive/MyDrive/Breast Cancer Project/IW/'+mag+'/A'
F = '/content/drive/MyDrive/Breast Cancer Project/IW/'+mag+'/F'
PT = '/content/drive/MyDrive/Breast Cancer Project/IW/'+mag+'/PT'
TA = '/content/drive/MyDrive/Breast Cancer Project/IW/'+mag+'/TA'
DC = '/content/drive/MyDrive/Breast Cancer Project/IW/'+mag+'/DC'
LC = '/content/drive/MyDrive/Breast Cancer Project/IW/'+mag+'/LC'
MC = '/content/drive/MyDrive/Breast Cancer Project/IW/'+mag+'/MC'
PC = '/content/drive/MyDrive/Breast Cancer Project/IW/'+mag+'/PC'

In [ ]:
dirlist=[A, F, PT, TA, DC, LC, MC, PC]
classes=['A','F', 'PT', 'TA', 'DC', 'LC', 'MC', 'PC']
# dirlist=[benign, malignant]
# classes=['B','M']
filepaths=[]
labels=[]
for i,j in zip(dirlist, classes):
    filelist=os.listdir(i)
    for f in filelist:
        filepath=os.path.join (i,f)
        filepaths.append(filepath)
        labels.append(j)
print ('filepaths: ', len(filepaths), '   labels: ', len(labels))

In [ ]:
Files=pd.Series(filepaths, name='filepaths')
Label=pd.Series(labels, name='labels')
df=pd.concat([Files,Label], axis=1)
df=pd.DataFrame(np.array(df).reshape(len(filepaths),8), columns = ['filepaths', 'labels'])
df.head()

In [ ]:
print(df['labels'].value_counts())

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, train_size=0.70, random_state=0)
#train_new, valid = train_test_split(train, train_size=0.90, random_state=0)

print(f"train set shape: {train.shape}")
print(f"test set shape: {test.shape}")
#print(f"validation set shape: {valid.shape}")

In [ ]:
# train = '/content/drive/MyDrive/fold5/40/train'
# test = '/content/drive/MyDrive/fold5/40/test'

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,
                                   shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, vertical_flip =True)
test_datagen = ImageDataGenerator(rescale = 1.0/255.)

In [ ]:
train_gen = train_datagen.flow_from_dataframe(dataframe=train,
                                              x_col = 'filepaths', y_col ='labels',
                                              target_size = (224,224), batch_size = 8,
                                              class_mode = 'categorical', shuffle = True)

test_gen = test_datagen.flow_from_dataframe(test,
                                            target_size = (224,224),   x_col = 'filepaths', y_col ='labels',
                                             class_mode = 'categorical',
                                            batch_size = 4, shuffle = False)

In [ ]:
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

def plotmodel(history,name):

    acc = history.history['acc']
    #val_acc = history.history['val_acc']
    loss = history.history['loss']
    #val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)

    plt.figure(1)
    plt.plot(epochs,smooth_curve(acc))
    #plt.plot(epochs,smooth_curve(val_acc))
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.legend(['train_acc'], loc='upper left')
    #plt.savefig('acc_'+name+'_'+mag+'_'+fold+'.png')

    plt.figure(2)
    plt.plot(epochs,smooth_curve(loss))
    #plt.plot(epochs,smooth_curve(val_loss))
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss'], loc='upper right')
   # plt.savefig('loss_'+name+'_'+mag+'_'+fold+'.png')

def label_smooth(y_true, y_pred):
    y_true=((1-0.1)*y_true+0.05)
    return K.categorical_crossentropy(y_true, y_pred)


In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    #plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 3.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="green" if cm[i, j] > thresh else "red", fontdict={'fontsize':'x-large'})

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
loss_fun= 'categorical_crossentropy'
gpu_num=1
k=5
lr1=0.005
lr2=0.0001
image_size=224
classes=2
ratio=8
fold='fold5'
mag='40'

In [ ]:
def train_model(model,train_gen,test_gen,mag,save_name,lr1,lr2,Epochs1,Epochs2):

    lr_decay=ReduceLROnPlateau(monitor='loss', factor=0.8, patience=3, verbose=1)
    save_model=ModelCheckpoint('effnetB6/'+mag+'/'+save_name+'/'+save_name+'{epoch:02d}.h5', monitor='loss',period=10,save_best_only=True)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=Adam(lr=1e-5,decay=1e-6),loss=loss_fun,metrics=['acc'])
    model.fit(train_gen,
                        #steps_per_epoch=train_num/batch_size,
                        #validation_data=test_data,
                        #validation_steps=test_num/batch_size,
                        epochs=Epochs1,
                        workers=4,
                        callbacks=[lr_decay,save_model])

    for layer in base_model.layers:
        layer.trainable = True

    model.compile(optimizer=Adam(lr=lr2,decay=0.00001),loss=loss_fun,metrics=['acc'])
    history = model.fit(train_gen,
                        #steps_per_epoch=train_num/batch_size,
                        #validation_data=test_data,
                        #validation_steps=test_num/batch_size,
                        epochs=Epochs2,
                        workers=4,
                        callbacks=[lr_decay,save_model])

    results =model.evaluate(test_gen)
    print('Test loss and accuracy: ',results)
    predictions = model.predict(test_gen)
    rounded_pred = np.argmax(predictions, axis=-1)
    cm = confusion_matrix(y_true=test_gen.classes, y_pred=rounded_pred)
    cm_plot_labels = ['A','F', 'PT', 'TA', 'DC', 'LC', 'MC', 'PC']
    plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')
    print(classification_report(y_true=test_gen.classes, y_pred=rounded_pred, target_names=cm_plot_labels))


    return history

In [ ]:
def stem(input,stride):
    x = Conv2D(32, 3, strides=stride, padding='same', activation='relu')(input)
    x = BatchNormalization()(x)
    x = Conv2D(32, 3, strides=1, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    return x

In [ ]:
def lka(input,fltr,dm=1):
    x = DepthwiseConv2D(5, strides=1, padding='same', depth_multiplier=dm, activation='relu')(input)
    x = BatchNormalization()(x)
    x = DepthwiseConv2D(7, strides=1, padding='same', depth_multiplier=dm, activation='relu',dilation_rate=3)(x)
    x = BatchNormalization()(x)
    x = Conv2D(fltr,1,padding='same',activation='relu')(x)
    x = Multiply()([x,input])
    # return x

In [ ]:
def lka(input,fltr,dm=1):
#     shape=K.int_shape(input)
#     q=Conv2D(fltr,1,padding='same',activation='relu')(input)
#     k=Conv2D(fltr,1,padding='same',activation='relu')(input)
#     v=Conv2D(fltr,1,padding='same',activation='relu')(input)
#   #q=Conv2D(fltr,3, padding="same", activation='relu')(li)
#   # dfc_li=Conv2D(1,1,padding='same',activation='relu')(li)
#   # k=v=DefConv_full(dfc_li, filters=fltr, kernel_size=3, strides=1, name='def_conv'+str(i)) # Key and Value
#   #   #print(q.shape)
#     Qshape=K.int_shape(q)
#     Kshape= K.int_shape(k)
#     Vshape= K.int_shape(v)
#     #print(Qshape,Kshape,Vshape)
#     a=Qshape[1]*Qshape[2]
#     q=Reshape((a,Qshape[3]))(q)
#     #print(q.shape)
#     k=Reshape((a,Kshape[3]))(k)
#     k=Permute((2,1))(k)
#     #print(k.shape)
#     qk=tf.matmul(q,k)
#     qk=Activation('softmax')(qk)
#     #print(qk.shape)
#     v=Reshape((a,Vshape[3]))(v)
#     #print(v.shape)
#     qkv=tf.matmul(qk,v)
#     #print(qkv.shape)
#     qkv=Activation('softmax')(qkv)
#     qkv=Reshape((Vshape[1],Vshape[2],Vshape[3]))(qkv)
  #print(qkv.shape)
  #return qkv
    x = DepthwiseConv2D(3, strides=1, padding='same', depth_multiplier=dm, activation='relu')(input)
    x = BatchNormalization()(x)
    x = DepthwiseConv2D(3, strides=1, padding='same', depth_multiplier=dm, activation='relu',dilation_rate=3)(x)
    x = BatchNormalization()(x)
    x = Conv2D(fltr,1,padding='same',activation='relu')(x)
    x = Multiply()([x,input])
    return x

In [ ]:
# def cascaded_lka_attn(input,fltr,nh):
#     attn = []
#     num_splits = nh
#     split_size = input.shape[2] // num_splits

#   # Split the tensor into smaller tensors along the channel dimension (axis=2)
#     feature_split = tf.split(input, num_or_size_splits= num_splits, axis=2)

#   # First head
#     x = Conv2D(fltr, 1, strides=1, padding='same', activation=tf.nn.gelu)(feature_split[0])
#     x = lka(x,fltr)
#     x = Conv2D(fltr, 1, strides=1, padding='same', activation=tf.nn.gelu)(x)
#     x = Add()([feature_split[0],x])
#     attn.append(x)

#     for i in range(1,nh):
#         x = Add()([feature_split[i],x])
#         x = Conv2D(fltr, 1, strides=1, padding='same', activation=tf.nn.gelu)(x)
#         x = lka(x,fltr)
#         x = Conv2D(fltr, 1, strides=1, padding='same', activation=tf.nn.gelu)(x)
#         x = Add()([feature_split[i],x])
#         attn.append(x)
#     mh_lka_attn = Concatenate(axis=-1)(attn)
#     mh_lka_attn = Conv2D(fltr,1, strides=1, padding='same', activation='relu')(mh_lka_attn)
#     return mh_lka_attn

In [ ]:
def lka_attn(input,fltr,nh):
    attn = []
    act = ['relu',tf.nn.gelu,'sigmoid']
    for i in range(nh):
        x = Conv2D(fltr, 1, strides=1, padding='same', activation=act[i])(input)
        x = lka(x,fltr)
        x = Conv2D(fltr, 1, strides=1, padding='same', activation=act[i])(x)
        attn.append(x)
    mh_lka_attn = Add()([attn[0],attn[1],attn[2]])
    return mh_lka_attn

In [ ]:
def trans_enc(input,fltr,nl,nh):
    x = input
    for i in range(nl):
        x1 = x
        x = LayerNormalization(epsilon=1e-6)(x1)
        x = lka_attn(x,fltr,nh)
        x = Add()([x1,x])
        y = LayerNormalization(epsilon=1e-6)(x)
        y = Conv2D(fltr, 1, strides=1, padding='same', activation='relu')(y)
        x = Add()([x,y])
    return x


In [ ]:
# def MSConv(input,fltr,strides,dm):
#     x = DepthwiseConv2D(3, strides=strides, padding='same', depth_multiplier=dm, activation='relu')(input)
#     x = BatchNormalization()(x)
#     shape = K.int_shape(x)

#     y = DepthwiseConv2D(3, strides=strides, padding='same', depth_multiplier=dm, activation='relu')(input)
#     y = BatchNormalization()(y)
#     y = DepthwiseConv2D(3, padding='same',  depth_multiplier=fltr, activation='relu')(y)
#     y = BatchNormalization()(y)

#     z = DepthwiseConv2D(3, strides=strides, padding='same', depth_multiplier=dm, activation='relu')(input)
#     z = BatchNormalization()(z)
#     z = DepthwiseConv2D(3,  padding='same', depth_multiplier=dm, activation='relu')(z)
#     z = BatchNormalization()(z)
#     z = DepthwiseConv2D(3, padding='same', depth_multiplier=dm, activation='relu')(z)
#     z = BatchNormalization()(z)

#     yr = tf.image.resize(y, size=[shape[1], shape[2]], method=tf.image.ResizeMethod.BILINEAR)
#     zr = tf.image.resize(z, size=[shape[1], shape[2]], method=tf.image.ResizeMethod.BILINEAR)

#     ot = Concatenate()([x,yr,zr])
#     ot = Conv2D(fltr, 1, padding='same', activation='relu')(ot)
#     ot = BatchNormalization()(ot)
#     inr = tf.image.resize(input, size=[ot.shape[1], ot.shape[2]], method=tf.image.ResizeMethod.BILINEAR)
#     out = Add()([inr,ot])

# #     out = DefConv_full(input, fltr,strides, kernel_size=3)
#     return out

In [ ]:
def MSConv(input,fltr,strides,dm):
    shape = K.int_shape(input)
    x = DepthwiseConv2D(3, strides=strides, padding='same', depth_multiplier=dm, activation='relu')(input)
    x = BatchNormalization()(x)

    y = DepthwiseConv2D(3, strides=strides, padding='same', depth_multiplier=dm, activation='relu')(x)
    y = BatchNormalization()(y)

    z = DepthwiseConv2D(3, strides=strides, padding='same', depth_multiplier=dm, activation='relu')(y)
    z = BatchNormalization()(z)

    z = Concatenate()([x,y,z])
    z = Conv2D(shape[3], 1, padding='same', activation='relu')(z)
    z = Add()([input,z])
    return z

In [ ]:
def GAB(inputs):
    shape=K.int_shape(inputs)
    x=AveragePooling2D(pool_size=(shape[1],shape[2])) (inputs)
    x=Conv2D(shape[3],1, padding='same') (x)
    x=Activation('relu') (x)
    x=Conv2D(shape[3],1, padding='same') (x)
    x=Activation('sigmoid') (x)
    C_A=Multiply()([x,inputs])

    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))  (C_A)
    x=Activation('sigmoid') (x)
    S_A=Multiply()([x,C_A])
    return S_A

In [ ]:
def defconv(input,fltr,strides,dm=1):
    x = BatchNormalization()(input)
    x = Conv2D(fltr, 1, padding='same', activation=tf.nn.gelu)(x)
    x = BatchNormalization()(x)
    x = MSConv(x,fltr,strides,dm)
    x = GAB(x)
    x = Conv2D(fltr, 1, padding='same', activation=tf.nn.gelu)(x)


    if strides==2:
        input = MaxPooling2D()(input)
        input = Conv2D(fltr, 1, padding='same', activation=tf.nn.gelu)(input)
        x = Add()([input,x])
    else:
        input = Conv2D(fltr, 1, padding='same', activation=tf.nn.gelu)(input)
        x = Add()([input,x])
    return x

In [ ]:
def Defvit_block(input, fltr,strides,nl,nh,dm=1):
    x = defconv(input,fltr,strides,dm)
    x = trans_enc(x,fltr,nl,nh)
    return x

In [ ]:
def build_defvit(input_shape,classes):
  # S-0 STEM stage
    input = Input(input_shape)
    x = stem(input,stride=2)
#     x = stem(x,stride=1)
    print('After stage-0',x.shape)

  # S-1 stage
    x = Defvit_block(x,32,strides=1,nl=2,nh=3)
    x = Defvit_block(x,32,strides=1,nl=2,nh=3)
    x = MaxPooling2D()(x)
    print('After stage-1',x.shape)

    # S-2 stage
    x = Defvit_block(x,64,strides=1,nl=2,nh=3)
    x = Defvit_block(x,64,strides=1,nl=2,nh=3)
    x = MaxPooling2D()(x)
    print('After stage-2',x.shape)

    # S-3 stage
    x = Defvit_block(x,128,strides=1,nl=4,nh=3)
    x = Defvit_block(x,128,strides=1,nl=4,nh=3)
    x = Defvit_block(x,128,strides=1,nl=4,nh=3)
    x = Defvit_block(x,128,strides=1,nl=4,nh=3)
    x = Defvit_block(x,128,strides=1,nl=4,nh=3)
    x = MaxPooling2D()(x)
    print('After stage-3',x.shape)

    # S-4 stage

    # x = Defvit_block(x,256,strides=1,nl=2,nh=3)
    # x = Defvit_block(x,256,strides=1,nl=2,nh=3)
    # x = MaxPooling2D()(x)
    # print('After stage-4',x.shape)


    x = GlobalAveragePooling2D()(x)
    out = Dense(classes,activation='softmax')(x)
    model = Model(input,out)
    return model



In [ ]:
model_vit = build_defvit((224,224,3),8)
model_vit.summary()

In [ ]:
lr_decay=ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=1)
model_vit.compile(optimizer=Adam(lr=0.001,decay=0.00001),loss=loss_fun,metrics=['acc'])
history = model_vit.fit(train_gen,
                        #steps_per_epoch=train_num/batch_size,
                        #validation_data=test_data,
                        #validation_steps=test_num/batch_size,
                        epochs=200,
                        workers=4,
                        callbacks=[lr_decay])

In [ ]:
results = model_vit.evaluate(test_gen)
print('Test loss and accuracy: ',results)
predictions = model_vit.predict(test_gen)
rounded_pred = np.argmax(predictions, axis=-1)
cm = confusion_matrix(y_true=test_gen.classes, y_pred=rounded_pred)
cm_plot_labels = ['B','M']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')
print(classification_report(y_true=test_gen.classes, y_pred=rounded_pred, target_names=cm_plot_labels))


In [ ]:
plotmodel(history,'defvit')